In [ ]:
from __future__ import division, print_function

# Third-party
from astropy.constants import G
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.optimize import leastsq

# Custom
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

### Observational constraints (taken from Oleg)

[Piffl et al. 2014](http://adsabs.harvard.edu/abs/2014A%26A...562A..91P) Mvir = (1.2-1.3)e12 Msun

[Deason et al. 2012](http://adsabs.harvard.edu/abs/2012MNRAS.425.2840D) [at 150 kpc] M = (5-10)e11 Msun

[Gnedin et al. 2010](http://adsabs.harvard.edu/abs/2010ApJ...720L.108G) [at 80 kpc] M = (6.9 +3.0-1.2)e11 Msun

[Deason et al. 2012](http://adsabs.harvard.edu/abs/2012MNRAS.424L..44D) [at 50 kpc] M = (4.2 ± 0.4)e11 Msun

[McMillan 2016](http://arxiv.org/abs/1608.00971) M_disk,stellar = 5.4e10, Mvir = 1.3e12 Msun

[Bovy & Rix 2013](http://adsabs.harvard.edu/abs/2013ApJ...779..115B) [at 4-9 kpc] M_disk = 5.3e10, M_disk,stellar = 4.6e10

[Nakanishi & Sofue](http://adsabs.harvard.edu/abs/2016PASJ...68....5N) M_gas = 8e9 Msun

[Bland-Hawthorn & Gerhard 2016](http://arxiv.org/abs/1602.07702) M_NSD = (1.4 ± 0.6)e9 Msun, r_h,NSD = 90 pc,
M_bulge = (1.4-1.7)e10 Msun, M_thin = (4 ± 1)e10, r_thin = 2.6 ± 0.5 kpc, M_thick = (8 ± 3)e9 Msun, r_thick = 2 ± 0.2 kpc, R_0 = 8.2 ± 0.1 kpc, V_0 = 238 ± 15 km/s, V_0/R_0 = 29.0 ± 1.8 km/s/kpc, M(8.2kpc) = 1.08e11 Msun

[Launhardt et all. 2002](http://adsabs.harvard.edu/abs/2002A%26A...384..112L) [at 120 pc] NSD mass = (8 ± 2)e8 Msun, scale radius = 90 pc

[Feldmeier et al. 2014](http://adsabs.harvard.edu/abs/2014A%26A...570A...2F) [at 10 pc] total mass = (3 ± 1)e7 Msun

[Chatzopoulos et al. 2015](http://adsabs.harvard.edu/abs/2015MNRAS.447..948C) [at 1 and 4 pc] R_0 = 8.33 ± 0.11 kpc, M_BH = (4.23 ± 0.14)e6 Msun, M_NSC = (1.8 ± 0.3)e7 Msun, r_h,NSC = 4.2 ± 0.4 pc, M_NSC(1pc) = 0.89e6 Msun

In [ ]:
robs =   np.array([ 0.001, 0.004, 0.010001,  0.12, 8.2,     50.,    80.,    150.  ])
Mobs =   np.array([ 5.1e6, 1.3e7, 2.6e7, 8.e8, 1.08e11, 4.2e11, 6.9e11, 9.0e11 ])
Mobs_l = np.array([ 4.6e6, 1.1e7, 1.6e7, 6.e8, 9.37e10, 3.8e11, 5.0e11, 5.0e11 ])
Mobs_u = np.array([ 5.6e6, 1.5e7, 3.6e7, 1.e9, 1.24e11, 4.6e11, 9.9e11, 1.1e12 ])

First my by-eye fit:

In [ ]:
# Background Milky Way potential
mw_potential = gp.CCompositePotential()
mw_potential['nucl'] =  gp.HernquistPotential(m=2E9, c=0.1, units=galactic)
mw_potential['bulge'] = gp.HernquistPotential(m=5E9, c=1., units=galactic)
mw_potential['disk'] = gp.MiyamotoNagaiPotential(m=6.8E10*u.Msun, a=3*u.kpc, b=280*u.pc, 
                                                 units=galactic)

# for DM halo potential
M_h = 6E11 * u.Msun
rs_h = 16. * u.kpc
v_c = np.sqrt(((np.log(2.) - 0.5) * (G * M_h / rs_h)).decompose(galactic).value)
mw_potential['halo'] = gp.SphericalNFWPotential(v_c=v_c, r_s=rs_h, units=galactic)

In [ ]:
r = np.logspace(-3.5, 2.6, 1024)
xyz = np.zeros((3,r.size))
xyz[0] = r
menc = mw_potential.mass_enclosed(xyz*u.kpc)

# observational points
plt.scatter(robs, Mobs, s=20, marker='s', color='k')
yerr1 = Mobs - Mobs_l
yerr2 = Mobs_u - Mobs
plt.errorbar(robs, Mobs, yerr=[yerr1,yerr2], ecolor='k', capthick=0, linestyle='None')

plt.loglog(r, menc.value, marker='')
plt.xlim(10**-3.5, 10**2.6)
plt.ylim(10**6.5, 10**12.25)

plt.xlabel('$r$ [kpc]')
plt.ylabel('$M(<r)$ [M$_\odot$]')

In [ ]:
mw_potential = gp.CCompositePotential()
mw_potential['bulge'] = gp.HernquistPotential(m=5E9, c=1., units=galactic)
mw_potential['disk'] = gp.MiyamotoNagaiPotential(m=6.8E10*u.Msun, a=3*u.kpc, b=280*u.pc, 
                                                 units=galactic)
    
def get_mw_potential(log_M_halo, r_s, log_M_nucl, a):
    # Background Milky Way potential
    mw_potential['nucl'] =  gp.HernquistPotential(m=np.exp(log_M_nucl), c=a, units=galactic)
    v_c = np.sqrt(((np.log(2.) - 0.5) * (G * np.exp(log_M_halo)*u.Msun / (r_s*u.kpc))).decompose(galactic).value)
    mw_potential['halo'] = gp.SphericalNFWPotential(v_c=v_c, r_s=r_s, units=galactic)
    return mw_potential

In [ ]:
fit_robs = robs[2:]
fit_Mobs = Mobs[2:]
log_Mobs = np.log(fit_Mobs)
x0 = (np.log(6E11), 16., np.log(2E9), 0.1)

xyz = np.zeros((3,fit_robs.size))

def f(p):
    pot = get_mw_potential(*p)
    
    xyz[0] = fit_robs
    log_menc = np.log(mw_potential.mass_enclosed(xyz).to(u.Msun).value)
    return (log_menc - log_Mobs) / 0.7

p_opt, ier = leastsq(f, x0=x0)
assert ier in range(1,4+1)

In [ ]:
p_opt

In [ ]:
fit_potential = get_mw_potential(*p_opt)

In [ ]:
r = np.logspace(-3.5, 2.6, 1024)
xyz = np.zeros((3,r.size))
xyz[0] = r
menc = fit_potential.mass_enclosed(xyz*u.kpc)

# observational points
fig,ax = plt.subplots(1,1,figsize=(6,6))

ax.scatter(robs, Mobs, s=20, marker='o', color='k')
yerr1 = Mobs - Mobs_l
yerr2 = Mobs_u - Mobs
ax.errorbar(robs, Mobs, yerr=[yerr1,yerr2], ecolor='k', capthick=0, linestyle='None')

ax.loglog(r, menc.value, marker='')
ax.set_xlim(10**-3.5, 10**2.6)
ax.set_ylim(10**6.5, 10**12.25)

ax.set_xlabel('$r$ [kpc]')
ax.set_ylabel('$M(<r)$ [M$_\odot$]')

fig.tight_layout()

fig.savefig("../paper/figures/mass-profile.pdf")

In [ ]:
np.exp(p_opt[0]), p_opt[1]

In [ ]:
np.exp(p_opt[2]), p_opt[3]

$$
M_H = 5.73 \times 10^{11}~Msun \\
r_H = 15.6~{\rm kpc} \\
M_N = 1.98 \times 10^{9}~Msun \\
r_N = 79~{\rm pc} 
$$